# Hyperparameter Optimization

TODO: update

## Google Colab

The first cell will only be run in Google Colab, the second one locally as well.

In [1]:
import sys

import numpy as np
from segmentation_models_pytorch.encoders import get_preprocessing_fn

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # noinspection PyUnresolvedReferences
    from google.colab import drive
    drive.mount('/content/drive')

In [2]:
import os
import glob

# let's keep this cell at the beginning for every notebook
# for more convenient training in Google Colab
def get_root_path(filename: str) -> str: 
    """Get root path based on notebook's name."""
    filepath = glob.glob(os.getcwd() + '/**/' + filename, recursive = True)[0]
    return os.path.dirname(os.path.dirname(filepath))

ROOT_PATH = get_root_path('backbones.ipynb')
sys.path.append(ROOT_PATH)

# go to the drive directory
os.chdir(ROOT_PATH) if IN_COLAB else None

## Imports

In [3]:
import os
import cv2
import torch

import albumentations as A
import pandas as pd
import segmentation_models_pytorch as smp
import flaml

from ray import tune
from scripts.evaluation import EvaluationMonitor, get_best_f1_per_setup
from scripts.preprocessing import RoadDataset, split_data
from scripts.training import train_model, setup_seed, tune_hyperparams
from scripts.plotting import plot_post_processing
from torch.utils.data import DataLoader, SubsetRandomSampler

In [ ]:
# necessary for downloading some of the models
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [130]:
SEED = 16
setup_seed(16)

## Data

In [131]:
# specify train directory
train_directory = os.path.join(ROOT_PATH, 'data', 'raw', 'train')

In [133]:
# define transformations
train_tf = A.Compose([
    A.Flip(p=0.25),
    A.Rotate(p=0.5, limit=180, border_mode=cv2.BORDER_CONSTANT, rotate_method="ellipse"),
    A.CoarseDropout(max_holes=8, max_height=50, max_width=50, min_holes=None, min_height=25, min_width=25)
])

valid_tf = A.Compose([A.Resize(height=608, width=608, always_apply=True)])

In [ ]:
image_path_train, image_path_val, mask_path_train, mask_path_val = split_data(train_directory, 0.1)
preprocess_input = get_preprocessing_fn('resnet18', pretrained='imagenet')

# get train and val dataset instances
train_dataset = RoadDataset(image_path_train, mask_path_train, train_tf, preprocess_input)
val_dataset = RoadDataset(image_path_val, mask_path_val, valid_tf, preprocess_input)

## Encode-Decoder Combinations



## Hyperparameters

Since our aim is to see, how different architectures influence the training, we fix the model, epochs and batch sizes.

## Cross-Validation

In [141]:
max_num_epoch = 150
criteria = [
    smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True),
    smp.losses.FocalLoss(smp.losses.BINARY_MODE)
]

config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "num_epochs": tune.loguniform(1, max_num_epoch),
    "batch_size": tune.randint(1, 5),    # log transformed with base 2
    "criterion": criteria
}

In [ ]:
time_budget_s = 3600     # time budget in seconds
gpus_per_trial = 0.5    # number of gpus for each trial; 0.5 means two training jobs can share one gpu
num_samples = 500       # maximal number of trials
np.random.seed(7654321)

In [ ]:
'https://github.com/microsoft/FLAML/blob/main/notebook/tune_pytorch.ipynb'

In [ ]:
import time
start_time = time.time()
result = flaml.tune.run(
    tune.with_parameters(tune_hyperparams), # TODO: add other pirams
    config=config,
    metric="loss",
    mode="min",
    low_cost_partial_config={"num_epochs": 1},
    max_resource=max_num_epoch,
    min_resource=1,
    scheduler="asha",  # need to use tune.report to report intermediate results in train_cifar 
    resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
    local_dir='logs/',
    num_samples=num_samples,
    time_budget_s=time_budget_s,
    use_ray=True)